#Setup Notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

drivepath = '/content/drive/My Drive/Columbia/3 Fall 22/RA/Detroit/'

In [ ]:
# Import packages
# To ignore unimporant system warnings
import warnings
warnings.filterwarnings("ignore")

# We will use Pandas, Numpy, and Matplotlib which is a package for visualization with Python
import pandas as pd
import numpy as np

# Load a required package 
# This is a library for accessing and parsing data through URLs
from urllib.parse import urlencode
import urllib.request, json 
from bs4 import BeautifulSoup # for web scraping
import fiona
import plotly.express as px
import geopandas as gpd
import seaborn as sns # visualization styling package
import plotly.graph_objects as go
import urllib, json
import os
import plotly
from IPython.display import Image, display
from shapely.geometry import Polygon
import time
from shapely.geometry import shape
import contextily as ctx
from google.colab import files
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline 
import networkx as nx #library supporting networks
import matplotlib.pyplot as plt
import seaborn as sns # visualization styling package
import folium
from folium.plugins import HeatMap

In [ ]:
from shapely.geometry import Polygon, LineString, MultiLineString, Point, GeometryCollection
from pyproj import Geod
from shapely import wkt
import os
from dotenv import load_dotenv
load_dotenv()
MAPS_API_KEY=os.getenv("MAPS_API_KEY")


#Load Data

In [ ]:
path = drivepath + 'Data/ZCTA Boundaries/tl_2021_us_zcta520/tl_2021_us_zcta520.shp'
zcta = gpd.read_file(path)
zcta = zcta[['ZCTA5CE20','geometry']]
zcta['ZCTA5CE20'] = zcta['ZCTA5CE20'].astype(int)

In [ ]:
path = drivepath + 'Data/Area of Interest - ZCTAs.csv'
dt = pd.read_csv(path)
dt.rename(columns={'ZCTA':'ZCTA5CE20', '"City" Name':'City Name'},inplace=True)

dt = dt.merge(zcta,on='ZCTA5CE20')
df = gpd.GeoDataFrame(dt, geometry='geometry')
df.head()

In [ ]:
df[df['ZCTA5CE20']==48083]

In [ ]:
df.geometry[56]

In [ ]:
from shapely.geometry import MultiPolygon, Polygon
#Bounds = MultiPolygon(Polygon(p.exterior) for p in df['geometry'].unary_union).convex_hull
Bounds = df.geometry[56]
Bounds

In [ ]:
import osmnx as ox

In [ ]:
G = ox.graph_from_polygon(Bounds, network_type='all_private')
ox.plot_graph(G)

In [ ]:
edges = ox.graph_to_gdfs(G, edges=True, nodes=False)

In [ ]:
edges

In [ ]:
edges = edges.to_crs({'init': 'epsg:6346'})

In [ ]:
import numpy as np
from shapely.geometry import LineString
from shapely.ops import unary_union

In [ ]:
print(edges.geometry.iloc[257])

In [ ]:
distance_delta = 12 #in meters
all_points = []
i = 0

for i in range(len(edges)):
  line = edges.geometry.iloc[i]
  distances = np.arange(0, line.length, distance_delta)
  points = [line.interpolate(distance) for distance in distances]
  if  len(line.boundary) < 2:
    all_points = all_points + points
  else:
    all_points = all_points + points + [line.boundary[1]]

In [ ]:
points = gpd.GeoDataFrame(geometry=gpd.GeoSeries(all_points, crs="EPSG:6346").to_crs({'init':'epsg:4326'})).drop_duplicates().reset_index().drop(columns={'index'})

In [ ]:
points.geometry.iloc[0].coords[0][0]

In [ ]:
points.geometry.iloc[0].coords[0][1]

In [ ]:
print(len(points))

In [ ]:
from math import cos
#Position, decimal degrees
lat = 42.0
lon = -83.0
Pi=22/7

#Earth’s radius, sphere
R=6378137

#offsets in meters
dn = 100
de = 100

#Coordinate offsets in radians
dLat = dn/R
dLon = de/(R*cos(Pi*lat/180))

#OffsetPosition, decimal degrees
latO = dLat * 180/Pi
lonO = dLon * 180/Pi 

In [ ]:
latO

In [ ]:
lonO

In [ ]:
# import urllib library
from urllib.request import urlopen

import re #regex

In [ ]:
def get_meta(lat, lon, api_key):
    url = "https://maps.googleapis.com/maps/api/streetview/metadata?location={0:},%20{1:}&key={2:}"
    response = urlopen(url.format(lat, lon))
    return url.format(lat, lon), json.loads(response.read())

In [ ]:
la = 42.54727682045162
lo = -83.18432208709729

In [ ]:
get_meta(la, lo, MAPS_API_KEY)[0]

In [ ]:
!pip install fake_useragent

In [ ]:
from selenium.webdriver.common.by import By
from fake_useragent import UserAgent

options = webdriver.ChromeOptions()

options.add_argument('--headless')
options.add_argument("--incognito")
options.add_argument("--nogpu")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1280,1280")
options.add_argument("--no-sandbox")
options.add_argument("--enable-javascript")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')

ua = UserAgent()
userAgent = ua.random

driver = webdriver.Chrome(options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": userAgent})

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

from selenium.webdriver.common.by import By
driver.implicitly_wait(5)

In [ ]:
wd.get('https://www.google.com/maps/@42.5472777,-83.1843046,3a,15y,196.83h,89.6t/data=!3m7!1e1!3m5!1sE71c0CE94Yv-635ySrLcbQ!2e0!5s20201001T000000!7i16384!8i8192')

In [ ]:
from time import sleep

In [ ]:
url = 'https://stackoverflow.com/questions/57941221/how-can-i-use-jquery-with-selenium-execute-script-method'
driver.get(url)

x = driver.execute_script("""var jquery_script = document.createElement('script'); 
jquery_script.src = 'https://ajax.googleapis.com/ajax/libs/jquery/1.7.1/jquery.min.js';
// jquery_script.onload = function(){var $ = window.jQuery; $("h1").wrap("<i></i>");};
jquery_script.onload = function(){
  var $ = window.jQuery; 
  $("h1").wrap("<i></i>");
};
document.getElementsByTagName('head')[0].appendChild(jquery_script);
return jquery_script""")

In [ ]:
x.text

In [ ]:
# Open the URL you want to execute JS
URL = 'https://www.google.com/maps/@42.5472777,-83.1843046,3a,15y,196.83h,89.6t/data=!3m7!1e1!3m5!1sE71c0CE94Yv-635ySrLcbQ!2e0!5s20201001T000000!7i16384!8i8192'
driver.get(URL)

sleep(5) # time to load jQuery library
#driver.execute_script('$ = window.jQuery;')

#sleep(6)
# Execute JS

#driver.execute_script("return $('.dryRY.uaSxsd').text()")

#driver.execute_script("console.log($('.dryRY.uaSxsd'))")

In [ ]:
driver.find_element

In [ ]:
fruit = driver.find_element(By.CSS_SELECTOR,"#fruits .tomatoes")
  

In [ ]:
element = driver.find_elements(By.CSS_SELECTOR,"#bottom-pane")

In [ ]:
element

In [ ]:
from selenium import webdriver
import undetected_chromedriver as uc
from time import sleep

driver_path = '/usr/local/bin/chromedriver.exe'

chromeOptions = uc.ChromeOptions() 
#chromeOptions = webdriver.ChromeOptions() 
chromeOptions.binary_location = '/Applications/Google Chrome Canary.app/Contents/MacOS/Google Chrome Canary'
#chromeOptions.executable_path = driver_path
#chromeOptions.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 1}) 
chromeOptions.add_argument("user-data-dir=/Users/kirthi/Library/Application Support/Google/Chrome Canary")
chromeOptions.add_argument("--no-sandbox") 
chromeOptions.add_argument("--disable-setuid-sandbox") 

chromeOptions.add_argument('--ignore-ssl-errors=yes')
chromeOptions.add_argument('--ignore-certificate-errors')
chromeOptions.add_experimental_option("useAutomationExtension", False)
chromeOptions.add_experimental_option("excludeSwitches",["enable-automation"])


#chromeOptions.add_argument("--remote-debugging-port=9515")  # this

#chromeOptions.add_argument("--disable-dev-shm-using") 
chromeOptions.add_argument("--disable-extensions") 
chromeOptions.add_argument("--disable-gpu") 
chromeOptions.add_argument("disable-infobars")

In [ ]:
from time import sleep

sleep(6)

doc = wd.execute_cdp_cmd(cmd="DOM.getDocument",cmd_args={})
doc_root_node_id = doc["root"]["nodeId"]
result = wd.execute_cdp_cmd(cmd="DOM.getOuterHTML",cmd_args={"nodeId":doc_root_node_id})
print (result['outerHTML'])

In [ ]:
!pip install PyChromeDevTools

In [ ]:
import requests

In [ ]:
# IMDB's homepage
imdb_url = 'https://www.google.com/maps/@42.5472777,-83.1843046,3a,15y,196.83h,89.6t/data=!3m7!1e1!3m5!1sE71c0CE94Yv-635ySrLcbQ!2e0!5s20201001T000000!7i16384!8i8192'

# Use requests to retrieve data from a given URL
imdb_response = requests.get(imdb_url)

# Parse the whole HTML page using BeautifulSoup
imdb_soup = BeautifulSoup(imdb_response.text, 'html.parser')

# Title of the parsed page
imdb_soup.title

In [ ]:
from selenium import webdriver  
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.common.keys import Keys  
from bs4 import BeautifulSoup

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

browser = webdriver.Chrome('chromedriver',options=chrome_options)
browser.get('https://www.google.com/maps/@42.5472777,-83.1843046,3a,75y,196.83h,89.6t/data=!3m7!1e1!3m5!1sE71c0CE94Yv-635ySrLcbQ!2e0!5s20201001T000000!7i16384!8i8192')  
html_source = browser.page_source  
browser.quit()

soup = BeautifulSoup(html_source,'html.parser')  
#class "postText" is not defined in the source code
comments = soup.findAll('div',{'class':'dryRY uaSxsd'})  
print (comments)

In [ ]:
!pip install pyshadow

In [ ]:
from pyshadow.main import Shadow
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

browser = webdriver.Chrome('chromedriver',options=chrome_options)
browser.get('https://www.google.com/maps/@42.5472777,-83.1843046,3a,75y,196.83h,89.6t/data=!3m7!1e1!3m5!1sE71c0CE94Yv-635ySrLcbQ!2e0!5s20201001T000000!7i16384!8i8192')
shadow = Shadow(browser)
browser.quit()

sleep(5)
wait = WebDriverWait(driver, 30)
sleep(5)
#wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#bottom-pane")))
#sleep(5)
element = shadow.find_elements("#bottom-pane")

In [ ]:
element.get_attribute('outerHTML')

In [ ]:
import PyChromeDevTools

chrome = PyChromeDevTools.ChromeInterface()
chrome.Network.enable()
chrome.Page.enable()
chrome.Page.navigate(url="http://www.stackoverflow.com/")
chrome.wait_event("Page.loadEventFired", timeout=60)

html_code=chrome.DOM.getOuterHTML()
print(html_code)

In [ ]:
# install chromium, its driver, and selenium
!apt update
!apt install chromium-chromedriver
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("window-size=1920,1080")
# open it, go to a website, and get results
wd = webdriver.Chrome(options=options)
wd.get("https://www.google.com")
print(wd.page_source)  # results

In [ ]:
(?<=" + labelBefore + “)(.*?)(?=” + labelAfter + ")

In [ ]:
filepath = 'sms-call-internet-mi-2013-11-08.txt'
path = os.path.join(drivepath,filepath)

In [ ]:
df = pd.read_csv(path, sep='\t', header=None)

In [ ]:
df = df.rename(columns={df.columns[0]: 'cellId', df.columns[1]: 'TimeInterval',df.columns[2]: 'SMSin',df.columns[3]: 'SMSout',df.columns[4]: 'Callin',df.columns[5]: 'Callout',df.columns[6]: 'InternetTraffic',df.columns[7]: 'CountryCode'})

In [ ]:
df = df.iloc[:,:-1].fillna(0)
df['TimeInterval'] = df['TimeInterval']/1000
df.head()

In [ ]:
df = df.groupby(['cellId','TimeInterval']).agg({'SMSin': 'sum', 'SMSout': 'sum', 'Callin': 'sum', 'Callout': 'sum', 'InternetTraffic': 'sum'}).reset_index()

In [ ]:
filepath = 'milano-grid.geojson'
path = os.path.join(drivepath,filepath)
grid = gpd.read_file(path)

In [ ]:
grid = grid.merge(df,on = 'cellId')

In [ ]:
grid.head()

In [ ]:
grid.shape

In [ ]:
grid['date'] = (grid['TimeInterval']+3600)
grid['date'] = pd.to_datetime(grid['date'],unit='s')
grid['date'] = grid['date'].astype(str)
grid[['D', 'T']] = grid['date'].str.split(' ', 1, expand=True)
grid['date'] = grid['D'] + 'T' + grid['T']+'.000Z'
grid = grid.iloc[:,:9]

#1970-01-01T00:00:00.000Z

grid.head()

In [ ]:
#1383926400000 = Callout - 5 PM - 5259
#1383909600000 = Callin - 12:20 PM - 4874
#1383901200000 = SMSin - 10 AM - 4963
#1383908400000 = SMSout - 12 PM - 5059
#1383928800000 = InternetTraffic - 5:40 PM - 5059

In [ ]:
dates = grid['date'].unique()

i = 0
SMSin_max = []
SMSout_max = []
Callin_max = []
Callout_max = []
Int_max = []

for i in range(len(dates)):
  x = grid.loc[grid[grid['date']==dates[i]]['SMSin'].idxmax()]['cellId']
  SMSin_max.append(x)

  x = grid.loc[grid[grid['date']==dates[i]]['SMSout'].idxmax()]['cellId']
  SMSout_max.append(x)

  x = grid.loc[grid[grid['date']==dates[i]]['Callin'].idxmax()]['cellId']
  Callin_max.append(x)

  x = grid.loc[grid[grid['date']==dates[i]]['Callout'].idxmax()]['cellId']
  Callout_max.append(x)

  x = grid.loc[grid[grid['date']==dates[i]]['InternetTraffic'].idxmax()]['cellId']
  Int_max.append(x)

  i = i+1

In [ ]:
max_df = pd.DataFrame()
max_df['date'] = pd.Series(dates)
max_df['SMSin'] = pd.Series(SMSin_max)
max_df['Callin'] = pd.Series(Callin_max)
max_df['SMSout'] = pd.Series(SMSout_max)
max_df['Callout'] = pd.Series(Callout_max)
max_df['InternetTraffic'] = pd.Series(Int_max)

In [ ]:
max_df['SMSin'].value_counts()

In [ ]:
max_df

In [ ]:
max_df['InternetTraffic'].value_counts()

In [ ]:
max_df[max_df['SMSin']==4857]

In [ ]:
max_df[max_df['SMSout']==7724]

In [ ]:
4857,7724,4874,5259,5059

In [ ]:
filename = 'Cleaned Data/spatialjoin_landuse_tele.pkl'
path = os.path.join(drivepath,filename)
lu_df = pd.read_pickle(path)  

In [ ]:
grid = grid.merge(lu_df[['cellId','code_2012']].groupby(['cellId'])['code_2012'].apply(lambda x: x.value_counts().index[0]).reset_index().merge(lu_df[['code_2012','class_2012']],on='code_2012').drop_duplicates(), on='cellId')

In [ ]:
grid

In [ ]:
filename = 'Cleaned Data/cells.csv'
path = os.path.join(drivepath,filename)
grid.to_csv(path)

In [ ]:
filename = 'Cleaned Data/max cells.csv'
path = os.path.join(drivepath,filename)
max_df.to_csv(path)

In [ ]:
filepath = 'social-pulse-milano.geojson'
path = os.path.join(drivepath,filepath)
sp = gpd.read_file(path)

In [ ]:
filepath = 'social-pulse-milano.txt'
path = os.path.join(drivepath,filepath)
#sp = gpd.read_file(path)

In [ ]:
#read input file
fin = open(path, "rt")
#read file contents to string
data = fin.read()
#replace all occurrences of the required string
data = data.replace('geomPoint.geom', 'geometry')
data = data.replace('SlicedDataCollection', 'FeatureCollection')

#close the input file
fin.close()
#open the input file in write mode
fin = open(path, "wt")
#overrite the input file with the resulting data
fin.write(data)
#close the file
fin.close()